<a href="https://colab.research.google.com/github/chiyeon01/CNN_Model_Mechanism/blob/main/pytorch/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchinfo

In [ ]:
# module import
import torch
from torchvision import models
import torch.nn as nn
import torchinfo
import albumentations as A

In [ ]:
class Config:
    batch_size = 8
    FineTune = False
    first_learning_rate = 1e-3
    second_learning_rate = 1e-4
    image_size = [224, 224]
    model_name = 'alexnet'
    make_summary = False,
    callbacks = []

In [ ]:
# 사전학습된 모델을 만드는 함수(create pretrained model function)
# 모든 가중치는 DEFAULT로 선언(all weights = 'DEFAULT')
def create_pretrained_model(model_name, make_summary=False):
    if model_name == 'alexnet':
        model = models.alexnet(weights='DEFAULT')
    elif model_name == 'resnet50':
        model = models.resnet50(weights='DEFAULT')
    elif model_name == 'resnet101':
        model = models.resnet101(weights='DEFAULT')
    elif model_name == 'efficientnet_b0':
        model = models.efficientnet_b0(weights='DEFAULT')
    elif model_name == 'efficientnet_b4':
        model = models.efficientnet_b4(weights='DEFAULT')
    elif model_name == 'efficientnet_v2_s':
        model = models.efficientnet_v2_s(weighs='DEFAULT')
    elif model_name == 'efficientnet_v2_m':
        model = models.efficientnet_v2_m(weighs='DEFAULT')
    elif model_name == 'efficientnet_v2_l':
        model = models.efficientnet_v2_l(weighs='DEFAULT')
    elif model_name == 'convnext_base':
        model = models.convnext_base(weights='DEFAULT')
    elif model_name == 'convnext_small':
        model = models.convnext_small(weights='DEFAULT')
    else:
        print("🧨🧨 [ERROR] 모델 이름이 존재하지 않습니다. 🧨🧨")
        return None

    if make_summary:
        print(torchinfo.summary(model, input_size=[1, 3] + Config.image_size,
                  col_names=['output_size', 'num_params', 'trainable'],
                  row_settings=['depth', 'var_names'],
                  depth=3))

    return model

In [ ]:
model = create_pretrained_model('resnet50', make_summary=Config.make_summary)

Layer (type (var_name):depth-idx)                  Output Shape              Param #                   Trainable
ResNet (ResNet)                                    [1, 1000]                 --                        True
├─Conv2d (conv1): 1-1                              [1, 64, 112, 112]         9,408                     True
├─BatchNorm2d (bn1): 1-2                           [1, 64, 112, 112]         128                       True
├─ReLU (relu): 1-3                                 [1, 64, 112, 112]         --                        --
├─MaxPool2d (maxpool): 1-4                         [1, 64, 56, 56]           --                        --
├─Sequential (layer1): 1-5                         [1, 256, 56, 56]          --                        True
│    └─Bottleneck (0): 2-1                         [1, 256, 56, 56]          --                        True
│    │    └─Conv2d (conv1): 3-1                    [1, 64, 56, 56]           4,096                     True
│    │    └─BatchNorm2d (bn